In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import datetime
import numpy as np


In [2]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df2 = viral[2]
group_cleaned = df2.drop([9])
group_cleaned.columns = ['Age Ranges of Confirmed Cases', 'Number_Cases', 'Number of Deaths']
date = datetime.datetime.today()
date_modify = str(date)
date_for_export = date_modify[0:10]
group_cleaned["Date"] = date_for_export
group_cleaned

,Age Ranges of Confirmed Cases,Number_Cases,Number of Deaths,Date
0,0-10,51,1,2020-04-09
1,11-20,218,0,2020-04-09
2,21-30,991,1,2020-04-09
3,31-40,747,1,2020-04-09
4,41-50,730,5,2020-04-09
5,51-60,845,9,2020-04-09
6,61-70,586,23,2020-04-09
7,71-80,288,24,2020-04-09
8,81+,156,30,2020-04-09


In [5]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df2 = viral[8]
TN_county = df2[:-3]
TN_county.columns = ['County', 'Positive', 'Negative', 'Death']
TN_county["County"]=TN_county["County"].str.split(" County", expand = True)
date = datetime.datetime.today()
date_modify=str(date)
date_to_add = date_modify[0:10]
TN_county["Date"]=date_to_add

C:\Users\clayf\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\clayf\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [3]:
to_update=pd.read_csv("C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/Synched/TN_cases_yesterday.csv")
synched=TN_county.append(to_update, sort=True)
TN_cases=synched.sort_values(["County", "Date"], ascending=True)
TN_cases["Duplicates"]=TN_cases["County"]+TN_cases['Date']
TN_cases = TN_cases.drop_duplicates(subset="Duplicates", keep="last")
TN_overtime=TN_cases.drop('Duplicates', axis=1)
TN_overtime.reset_index(drop=True, inplace=True)
TN_overtime

,County,Date,Death,Negative,Percent of County Tested,Percentage of County with Coronavirus,Population,Positive
0,Anderson,2020-03-20,0.0,NaN,NaN,0.001,76978.0,1.0
1,Anderson,2020-03-21,0.0,NaN,NaN,0.001,76978.0,1.0
2,Anderson,2020-03-22,0.0,NaN,NaN,0.001,76978.0,1.0
3,Anderson,2020-03-23,0.0,NaN,NaN,0.001,76978.0,1.0
4,Anderson,2020-03-24,0.0,NaN,NaN,0.001,76978.0,1.0
...,...,...,...,...,...,...,...,...
1583,Wilson,2020-04-05,NaN,961.0,0.7,0.064,144657.0,93.0
1584,Wilson,2020-04-06,NaN,996.0,0.8,0.067,144657.0,97.0
1585,Wilson,2020-04-07,NaN,1092.0,0.8,0.078,144657.0,113.0
1586,Wilson,2020-04-08,NaN,1168.0,0.9,0.081,144657.0,117.0


In [7]:
TNpop = pd.read_csv("C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/Population Estimates by County.csv")
TNpop= TNpop.drop([96])
TNpop = TNpop.drop([0])
TNpop["Population Estimates by County"]= TNpop["Population Estimates by County"].str.split(" County", n = 1, expand = True)
TNpop = TNpop.rename(columns={'Population Estimates by County': 'County'})
TN_county_populations = TNpop.rename(columns={'Unnamed: 1': 'Population'})
TN_county_populations["Population"] = TN_county_populations["Population"].str.replace(",","").astype(int)
TN_county_populations

,County,Population
1,Anderson,76978
2,Bedford,49713
3,Benton,16160
4,Bledsoe,15064
5,Blount,133088
...,...,...
91,Wayne,16673
92,Weakley,33328
93,White,27345
94,Williamson,238412


In [19]:
TN_data = TN_county_populations.merge(TN_overtime,left_on='County',right_on='County', how='left')
TN_data=TN_data.drop('Population_y', axis=1)
TN_data=TN_data.rename(columns={'Population_x':'Population'})
TN_data

,County,Population,Date,Death,Negative,Percent of County Tested,Percentage of County with Coronavirus,Positive
0,Anderson,76978,2020-03-20,0.0,NaN,NaN,0.001,1.0
1,Anderson,76978,2020-03-21,0.0,NaN,NaN,0.001,1.0
2,Anderson,76978,2020-03-22,0.0,NaN,NaN,0.001,1.0
3,Anderson,76978,2020-03-23,0.0,NaN,NaN,0.001,1.0
4,Anderson,76978,2020-03-24,0.0,NaN,NaN,0.001,1.0
...,...,...,...,...,...,...,...,...
1581,Wilson,144657,2020-04-05,NaN,961.0,0.7,0.064,93.0
1582,Wilson,144657,2020-04-06,NaN,996.0,0.8,0.067,97.0
1583,Wilson,144657,2020-04-07,NaN,1092.0,0.8,0.078,113.0
1584,Wilson,144657,2020-04-08,NaN,1168.0,0.9,0.081,117.0


In [18]:
TN_data["Percentage of County with Coronavirus"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data=TN_data.sort_values("Positive", ascending=False).reset_index(drop=True)
TN_data["Percent of County Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data.head(60)

,County,Population,Date,Death,Negative,Percent of County Tested,Percentage of County with Coronavirus,Positive
0,Shelby,937166,2020-04-09,20.0,4727.0,0.6,0.107,1006.0
1,Davidson,694144,2020-04-09,13.0,6812.0,1.1,0.145,1004.0
2,Davidson,694144,2020-04-08,13.0,6588.0,1.1,0.136,946.0
3,Shelby,937166,2020-04-08,17.0,4116.0,0.5,0.097,912.0
4,Davidson,694144,2020-04-07,9.0,6359.0,1.0,0.128,888.0
5,Shelby,937166,2020-04-07,15.0,3589.0,0.5,0.089,835.0
6,Davidson,694144,2020-04-06,7.0,5979.0,1.0,0.118,819.0
7,Davidson,694144,2020-04-05,6.0,5719.0,0.9,0.115,801.0
8,Shelby,937166,2020-04-06,13.0,3396.0,0.4,0.082,766.0
9,Davidson,694144,2020-04-04,6.0,5445.0,0.9,0.107,741.0


In [ ]:
# TNY='https://en.wikipedia.org/wiki/User:Michael_J/County_table'
# stuff=pd.read_html(TNY)
# TN_land_data=stuff[0]
# TN_land_data.to_csv('C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/TN_land_data.csv')
land_data=pd.read_csv('C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/TN_land_data.csv')
TN_lat_long=land_data[land_data['State'] == 'TN']

lat=TN_lat_long.iloc[:,13]
long=TN_lat_long.iloc[:,14]
lats=[]
longs=[]
for i in lat:
    x=i[:-1]
  
    
    lats.append(x)

for steve in long:
    y=steve[:-1]
  
    longs.append(y)
print(lats)


TN_lat_long['Latitude']=lats
TN_lat_long["Longitude"]=longs


TN_lat_long.reset_index(drop=True, inplace=True)
TN_lat_long.drop(["Unnamed: 0", "Sort [1]"],axis=1)
TN_lat_long["Region"] = 'US-TN'
TN_lat_long["Country"] = 'United States'
TN_lat_long

TN_lat_long.to_csv('C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/TN_geo_data.csv', index=False)

In [ ]:
TN_overtime["Percentage of County Population"] = round((TN_overtime["Positive"]/TN_overtime["Population"])*100,3)
TN_overtime=TN_overtime.sort_values("Positive", ascending=False).reset_index(drop=True)
TN_overtime["Percent_Tested"] = round(((TN_overtime["Positive"]+TN_overtime["Negative"])/TN_overtime["Population"])*100,1)
TN_overtime

In [ ]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df2 = viral[5]
TN_county = df2[:-3]
headers=TN_county.columns.values.tolist()
TN_county["County"]=TN_county["County"].str.split(" County", expand = True)
TN_county.columns = ['County', 'Positive', 'Negative', 'Death']
TN_county


In [ ]:
date = datetime.datetime.today()
date_modify=str(date)
date_to_add = date_modify[0:10]
TN_county["Date"]=date_to_add

In [ ]:
TNpop = pd.read_csv("Population Estimates by County.csv")
TNpop= TNpop.drop([96])
TNpop = TNpop.drop([0])
TNpop["Population Estimates by County"]= TNpop["Population Estimates by County"].str.split(" County", n = 1, expand = True)
TNpop = TNpop.rename(columns={'Population Estimates by County': 'County'})
TN_county_populations = TNpop.rename(columns={'Unnamed: 1': 'Population'})
TN_county_populations["Population"] = TN_county_populations["Population"].str.replace(",","").astype(int)
TN_county_populations


In [ ]:
TN_data = TN_county_populations.merge(TN_county,left_on='County',right_on='County', how='left')
TN_data["Percentage of County Population"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data=TN_data.sort_values("Positive", ascending=False).reset_index(drop=True)
TN_data["Percent_Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data

In [ ]:
to_update=pd.read_csv("C:/Users/clayf/Documents/Coronavirus/Covid_predictions/TN/Resources/Synched/TN_cases_yesterday.csv")
synched=TN_data.append(to_update, sort=True)
TN_cases=synched.sort_values(["County", "Date"], ascending=True)
TN_cases["Duplicates"]=TN_cases["County"]+TN_cases['Date']
TN_cases = TN_cases.drop_duplicates(subset="Duplicates", keep="last")
TN_overtime=TN_cases.drop('Duplicates', axis=1)
TN_overtime=TN_overtime.drop('index', axis=1)
TN_overtime.reset_index(drop=True, inplace=True)
TN_overtime


In [ ]:
TN_overtime.sort_values(["County", "Date"], ascending=True)



## 

In [ ]:
TN_data["Percentage of County Population"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data.reset_index(drop=True)
TN_data["Percent_Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data=TN_data.sort_values("Percent_Tested", ascending=False)
TN_data=TN_data
TN_data


In [ ]:
url2="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
viral_times=pd.read_csv(url2)
TN=viral_times[viral_times["state"] == 'Tennessee']
# TN=TN[TN["date"] == '2020-03-31']
TN=TN.rename(columns={'county':'County'})
TN=TN.rename(columns={'date':'Date'})
TN=TN.rename(columns={'cases':'Positive'})
TN=TN.rename(columns={'deaths':'Deaths'})
TN=TN.drop(['state'], axis=1)
TN

In [ ]:
TN_deaths = TN_data.merge(TN,left_on='County',right_on='County', how='left')
TN_deaths.head(60)

df_counties=viral[4]
file = datetime.datetime.today()
date = file[0:10]
date


In [ ]:
# from pathlib import Path
stuff=pd.read_csv("C:/Users/clayf/Documents/VU-NSH-DATA-PT-07-2019-U-C/Activities/Week3/02/09-Ins_WriteCSV/output/new.csv")
stuff["crapola"]= 2
stuff.to_csv("C:/Users/clayf/Documents/VU-NSH-DATA-PT-07-2019-U-C/Activities/Week3/02/09-Ins_WriteCSV/output/new.csv", index=False)


In [ ]:
US = "https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fcases-in-us.html"
us_data = pd.read_html(US)
us_total_cases = us_data[0]
us_total_cases.rename(columns={list(us_total_cases)[1]:'Total'}, inplace=True)
us_total_cases['Total']=us_total_cases['Total'].astype(int)
US = us_total_cases.iloc[3,1]

In [ ]:
date = datetime.datetime.today()
date_modify = str(date)
date_to_add = date_modify[0:10]
x['date'] = date_to_add



In [ ]:
TN_ratio = (TN/6700000)*100
US_ratio = (US/372000000)*100

print(TN_ratio)
print(US_ratio)